# For sample learing
## 1. single cell agent, custom environment, custom loop with Q table (value based)

In [1]:
import sys
import os

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from envs.minimumEnv_singleCell import minimumColonyEnv
import numpy as np
import random

In [ ]:
# Q-learning method
# implement Q-table
Q = {}
for n in [0,1,2]:
    for p in [0,1,2]:
        Q[(n,p)] = [0,0,0]  # 3 actions

alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
epsilon = 0.1  # exploration rate

env = minimumColonyEnv()
num_episodes = 500

for episode in range(num_episodes):
    state = env.reset()
    for t in range(20):
        # epsilon-greedy action selection
        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, 2)  # explore
        else:
            action = np.argmax(Q[state])  # exploit
            
        next_state, reward, done = env.step(action)
        
        # Q-learning update
        #print(state, " : ", Q)
        best_next = max(Q[next_state])
        Q[state][action] = Q[state][action] + alpha * (reward + gamma * best_next - Q[state][action])
        
        state = next_state

In [3]:
for state, actions in Q.items():
    best = np.argmax(actions)
    print(f"State {state}: best action {best} (Q={actions})")

State (0, 0): best action 2 (Q=[7.213021139647562, 6.1619250720739736, 8.971743736164308])
State (0, 1): best action 2 (Q=[7.489111172475347, 6.461366888753406, 9.586523835541822])
State (0, 2): best action 2 (Q=[7.38309940846315, 6.8336835757242556, 9.062064742811907])
State (1, 0): best action 0 (Q=[10.872861886349263, 6.5420771441203955, 7.78888145610285])
State (1, 1): best action 0 (Q=[11.845432865893033, 6.727180372290738, 8.014759464678434])
State (1, 2): best action 2 (Q=[7.16056082771801, 6.411007252464571, 9.074740317583512])
State (2, 0): best action 1 (Q=[11.022480814508775, 12.108206071762718, 8.357044802264127])
State (2, 1): best action 0 (Q=[10.991240865283928, 6.454442287063826, 8.261927115876809])
State (2, 2): best action 2 (Q=[7.5184592655859515, 6.893539675943709, 9.138608284266493])


## 2. policy based

**Goal**: Parameterize the policy, π(a | s; θ), by a set of weights θ (e.g., a neural network). The network takes the state s as input and outputs a probability distribution over actions.

**How it Works**: The agent runs a batch of episodes (e.g., a full simulation of colony growth). It then tweaks the policy parameters θ to make good actions (those that led to high total reward in the episode) more probable, and bad actions less probable.
[REINFORCE Algorithm](https://dilithjay.com/blog/reinforce-a-quick-introduction-with-code)

The core update is: θ_new ← θ_old + α * ∇J(θ)
Where ∇J(θ) is the policy gradient, an estimate of the direction in parameter space that will increase the expected reward.

In [4]:
from agents.manualPolicyGradient import manualPolicyGradient
policy = manualPolicyGradient()

# Training with policy gradients
env = minimumColonyEnv()
num_episodes = 1000
learning_rate = 0.1

for episode in range(num_episodes):
    state = env.reset()
    episode_states = []
    episode_actions = []
    episode_rewards = []
    episode_probs = []
    
    # Run episode
    for t in range(20):
        action, action_probs = policy.choose_action(state)
        next_state, reward, done = env.step(action)
        
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        episode_probs.append(action_probs)
        
        state = next_state
    
    # Update policy after episode
    policy.update(episode_states, episode_actions, episode_rewards, learning_rate)
    
    # Print progress
    if episode % 200 == 0:
        total_reward = sum(episode_rewards)
        print(f"Episode {episode}, Total Reward: {total_reward:.2f}")

# Display learned policy
print("\nLearned Policy (action probabilities):")
for state in [(n, p) for n in [0,1,2] for p in [0,1,2]]:
    probs = policy.get_action_probs(state)
    best_action = np.argmax(probs)
    print(f"State {state}: P(actions) = {[f'{p:.3f}' for p in probs]}, best action: {best_action}")

# Test the learned policy
print("\nTesting learned policy:")
state = env.reset()
total_test_reward = 0
for t in range(10):
    action, probs = policy.choose_action(state)
    next_state, reward, done = env.step(action)
    print(f"State {state} -> Action {action} -> Reward {reward} -> Next state {next_state}")
    total_test_reward += reward
    state = next_state
print(f"Total test reward: {total_test_reward}")

Episode 0, Total Reward: -13.00
Episode 200, Total Reward: 20.00
Episode 400, Total Reward: 21.00
Episode 600, Total Reward: 21.00
Episode 800, Total Reward: 26.00

Learned Policy (action probabilities):
State (0, 0): P(actions) = ['0.005', '0.001', '0.993'], best action: 2
State (0, 1): P(actions) = ['0.006', '0.006', '0.988'], best action: 2
State (0, 2): P(actions) = ['0.005', '0.002', '0.993'], best action: 2
State (1, 0): P(actions) = ['0.994', '0.002', '0.004'], best action: 0
State (1, 1): P(actions) = ['0.996', '0.002', '0.002'], best action: 0
State (1, 2): P(actions) = ['0.004', '0.002', '0.994'], best action: 2
State (2, 0): P(actions) = ['0.006', '0.991', '0.003'], best action: 1
State (2, 1): P(actions) = ['0.996', '0.001', '0.003'], best action: 0
State (2, 2): P(actions) = ['0.006', '0.003', '0.991'], best action: 2

Testing learned policy:
State (1, 0) -> Action 0 -> Reward 2 -> Next state (1, 0)
State (1, 0) -> Action 0 -> Reward 2 -> Next state (1, 1)
State (1, 1) -> 

### 2.1 Key Differences from Value-Based Approach:  
No Q-table: Instead of storing action values, we store policy parameters theta that define action probabilities.

- Action Selection:

    - Value-based: action = argmax(Q[state]) (deterministic based on values)

    - Policy-based: action = sample from π(a|state) (stochastic based on probabilities)

- Learning Mechanism:

    - Value-based: Directly update Q-values using Bellman equation

    - Policy-based: Update policy parameters in the direction that increases expected reward

- Output:

    - Value-based: Shows which action has highest value in each state

    - Policy-based: Shows probability distribution over actions for each state

#### Advantages for Colony Project:
1. Natural Stochasticity: Bacteria don't always make the same decision in the same state - policy gradients capture this biological reality.

2. Continuous Action Spaces: Easy to extend to continuous actions (like growth rate).

3. Better Credit Assignment: Directly links actions to long-term outcomes through the episode returns.

4. Multi-agent Friendly: Each agent can have its own stochastic policy, which often works better in multi-agent settings.

### 2.2 why logits
logit p = $\sigma^{-1}(p) = ln \frac{p}{1-p}$ for $ p \in (0,1)$ 

#### Why Use Logits Instead of Probabilities?
1. Numerical Stability in Gradient Updates:  
If we stored probabilities directly, the gradient update would be messy: We'd have to ensure probabilities stay positive and sum to 1. This requires complex constrained optimization

2. Avoid Probability Boundary Issues:  
If we stored probabilities and one probability became very small (say 0.001), it would be hard to "recover" that action if it later became good. With logits, even if the softmax probability is tiny, the logit can still be updated freely.

3. Better Gradient Behavior:  
The gradient of the log probability with respect to logits has a nice property:  
$\nabla log \pi(a|s) = (1 - π) \text{for chosen action, } -\pi $ for others  
This gradient is well-behaved and doesn't vanish when probabilities approach 0 or 1.


In [1]:
import numpy as np

class PolicyWithLogits:
    def __init__(self):
        self.theta = {}
        for n in [0, 1, 2]:
            for p in [0, 1, 2]:
                self.theta[(n, p)] = [0.0, 0.0, 0.0]  # Logits
    
    def get_action_probs(self, state):
        """Convert logits to probabilities using softmax"""
        logits = self.theta[state]
        exp_logits = np.exp(logits - np.max(logits))  # Numerical stability
        probs = exp_logits / np.sum(exp_logits)
        return probs

class PolicyWithDirectProbs:
    def __init__(self):
        self.theta = {}
        for n in [0, 1, 2]:
            for p in [0, 1, 2]:
                self.theta[(n, p)] = [0.33, 0.33, 0.34]  # Direct probabilities
    
    def get_action_probs(self, state):
        """Already have probabilities, just return them"""
        return self.theta[state]

# Compare the two approaches
policy_logits = PolicyWithLogits()
policy_direct = PolicyWithDirectProbs()

state = (1, 1)
print("With Logits:")
print(f"Initial logits: {policy_logits.theta[state]}")
print(f"Probabilities: {policy_logits.get_action_probs(state)}")

print("\nWith Direct Probabilities:")
print(f"Probabilities: {policy_direct.get_action_probs(state)}")

# After an update that strongly prefers action 0:
print("\n--- After preferring action 0 ---")

# With logits - can represent strong preferences
policy_logits.theta[state] = [5.0, 0.0, 0.0]  # Strong preference for action 0
print(f"Logits: {policy_logits.theta[state]}")
print(f"Probabilities: {policy_logits.get_action_probs(state)}")

# With direct probabilities - limited by boundaries
policy_direct.theta[state] = [0.98, 0.01, 0.01]  # Can't go much higher
print(f"Probabilities: {policy_direct.get_action_probs(state)}")

With Logits:
Initial logits: [0.0, 0.0, 0.0]
Probabilities: [0.33333333 0.33333333 0.33333333]

With Direct Probabilities:
Probabilities: [0.33, 0.33, 0.34]

--- After preferring action 0 ---
Logits: [5.0, 0.0, 0.0]
Probabilities: [0.98670329 0.00664835 0.00664835]
Probabilities: [0.98, 0.01, 0.01]


### 2.3 Why softmax not simple conversion to probabilities
softmax: $\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}} $  
Why simple conversion fails:
- Logits can be negative - probabilities would become negative (invalid!)
- No amplification of differences - a logit of [1, 2, 3] would give similar probabilities to [100, 200, 300]
- No probabilistic interpretation - the outputs don't represent true probabilities

Why softmax:
1. Amplifies Relative Differences

2. Perfect for Gradient-Based Learning
The gradient of softmax has a beautiful property for policy gradients
The gradient ∇logπ(a|s) becomes very simple:  
For chosen action: (1 - π)  
For other actions: (-π)  
This makes the update rule clean and efficient

3. Interpretable as Probabilities


### 2.4.1 more on REINFORCE algorithm
[REINFORCE algorithm — Reinforcement Learning from scratch in PyTorch](https://medium.com/@sofeikov/reinforce-algorithm-reinforcement-learning-from-scratch-in-pytorch-41fcccafa107) 

#### 1. Policy Representation (by deepseek)

The policy is parameterized by $\theta$ and defines a probability distribution over actions:

$ \pi(a|s; \theta) = softmax(\theta[s,a']) = \frac{\exp(\theta[s,a])}{\sum_{a'} \exp(\theta[s,a'])} $

Where:
- $\pi(a|s; \theta)$ is the probability of taking action $a$ in state $s$
- $\theta[s,a]$ are the logits for state $s$ and action $a$

#### 2. Objective Function

We want to maximize the expected return:

$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \gamma^t R_t \right]
$

Where:
- $\gamma \in [0,1]$ is the discount factor
- $R_t$ is the reward at time $t$
- $\tau = (s_0, a_0, R_1, s_1, a_1, R_2, \ldots)$ is a trajectory

#### 3. Policy Gradient Theorem

The gradient of the objective is:

$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi(a_t|s_t; \theta) \cdot G_t \right]
$

Where:
- $G_t = \sum_{k=t}^T \gamma^{k-t} R_{k+1}$ is the discounted return from time $t$
- $\nabla_\theta \log \pi(a_t|s_t; \theta)$ is the score function

#### 4. Score Function Gradient

For the softmax policy, the gradient has a simple form:

$
\nabla_\theta \log \pi(a|s; \theta) = \mathbf{1}_a - \pi(s; \theta)
$

Where:
- $\mathbf{1}_a$ is a one-hot vector with 1 at position $a$ and 0 elsewhere
- $\pi(s; \theta)$ is the vector of action probabilities in state $s$

#### 5. Monte Carlo Estimate

We approximate the expectation by sampling $N$ trajectories:

$
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N \sum_{t=0}^T \left[ \mathbf{1}_{a_t^i} - \pi(s_t^i; \theta) \right] \cdot G_t^i
$

#### 6. Parameter Update Rule

$
\theta \leftarrow \theta + \alpha \cdot \nabla_\theta J(\theta)
$

Where $\alpha$ is the learning rate.

#### 7. Complete REINFORCE Algorithm

For each episode $i$:
1. Generate trajectory: $\tau^i = \{(s_0^i, a_0^i, R_1^i), (s_1^i, a_1^i, R_2^i), \ldots, (s_T^i, a_T^i, R_{T+1}^i)\}$
2. For $t = 0$ to $T$:
   $
   G_t^i = \sum_{k=t}^T \gamma^{k-t} R_{k+1}^i
   $
3. For $t = 0$ to $T$:
   $
   \theta \leftarrow \theta + \alpha \cdot \gamma^t \cdot G_t^i \cdot \left[ \mathbf{1}_{a_t^i} - \pi(s_t^i; \theta) \right]
   $

#### 8. With Return Normalization

To reduce variance, we normalize returns:

$
\hat{G}_t^i = \frac{G_t^i - \mu_G}{\sigma_G + \epsilon}
$

Where:
- $\mu_G = \frac{1}{N \cdot T} \sum_{i=1}^N \sum_{t=0}^T G_t^i$
- $\sigma_G = \sqrt{\frac{1}{N \cdot T} \sum_{i=1}^N \sum_{t=0}^T (G_t^i - \mu_G)^2}$
- $\epsilon = 10^{-8}$ (small constant for numerical stability)

#### 9. Per-State Update Rule

For a specific state-action pair $(s, a)$:

$
\theta[s,a] \leftarrow \theta[s,a] + \alpha \cdot \hat{G}_t \cdot (1 - \pi(a|s; \theta))
$

For other actions $a' \neq a$:

$
\theta[s,a'] \leftarrow \theta[s,a'] + \alpha \cdot \hat{G}_t \cdot (0 - \pi(a'|s; \theta))
$

#### 10. Learning Interpretation

- **When $\hat{G}_t > 0$**: Increase $\pi(a_t|s_t)$, decrease $\pi(a'|s_t)$ for $a' \neq a_t$
- **When $\hat{G}_t < 0$**: Decrease $\pi(a_t|s_t)$, increase $\pi(a'|s_t)$ for $a' \neq a_t$
- **Update magnitude**: Proportional to $|\hat{G}_t|$

### 2.4.2 Derivation of Policy Gradient Update Rule

##### Step 1: Objective Function

We want to maximize the expected return:

$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \gamma^t R(s_t, a_t) \right]
$

Where a trajectory $\tau = (s_0, a_0, s_1, a_1, \ldots, s_T)$ has probability:

$
P(\tau|\theta) = P(s_0) \prod_{t=0}^T \pi(a_t|s_t; \theta) P(s_{t+1}|s_t, a_t)
$

##### Step 2: Express Expectation Explicitly

$
J(\theta) = \sum_\tau P(\tau|\theta) \left( \sum_{t=0}^T \gamma^t R(s_t, a_t) \right)
$

##### Step 3: Take Gradient

$
\nabla_\theta J(\theta) = \nabla_\theta \left[ \sum_\tau P(\tau|\theta) \left( \sum_{t=0}^T \gamma^t R(s_t, a_t) \right) \right]
$

$
= \sum_\tau \nabla_\theta P(\tau|\theta) \left( \sum_{t=0}^T \gamma^t R(s_t, a_t) \right)
$

##### Step 4: Log-Derivative Trick

Use the identity: $\nabla_\theta P(\tau|\theta) = P(\tau|\theta) \nabla_\theta \log P(\tau|\theta)$

$
\nabla_\theta J(\theta) = \sum_\tau P(\tau|\theta) \nabla_\theta \log P(\tau|\theta) \left( \sum_{t=0}^T \gamma^t R(s_t, a_t) \right)
$

##### Step 5: Expand Log-Probability of Trajectory

$
\log P(\tau|\theta) = \log P(s_0) + \sum_{t=0}^T \left[ \log \pi(a_t|s_t; \theta) + \log P(s_{t+1}|s_t, a_t) \right]
$

$
\nabla_\theta \log P(\tau|\theta) = \sum_{t=0}^T \nabla_\theta \log \pi(a_t|s_t; \theta)
$

The dynamics terms $P(s_{t+1}|s_t, a_t)$ and initial state $P(s_0)$ don't depend on $\theta$.

##### Step 6: Substitute Back

$
\nabla_\theta J(\theta) = \sum_\tau P(\tau|\theta) \left( \sum_{t=0}^T \nabla_\theta \log \pi(a_t|s_t; \theta) \right) \left( \sum_{k=0}^T \gamma^k R(s_k, a_k) \right)
$

##### Step 7: Policy Gradient Theorem

We can rewrite this as:

$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi(a_t|s_t; \theta) \left( \sum_{k=t}^T \gamma^k R(s_k, a_k) \right) \right]
$

Or more commonly:

$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi(a_t|s_t; \theta) \cdot G_t \right]
$

Where $G_t = \sum_{k=t}^T \gamma^{k-t} R_{k+1}$ is the return from time $t$.

##### Step 8: Softmax Policy Gradient

For a softmax policy:

$
\pi(a|s; \theta) = \frac{\exp(\theta[s,a])}{\sum_{a'} \exp(\theta[s,a'])}
$

The gradient of the log-policy is:

$
\nabla_\theta \log \pi(a|s; \theta) = \frac{\nabla_\theta \pi(a|s; \theta)}{\pi(a|s; \theta)}
$

Compute the gradient for a specific parameter $\theta[s,a]$:

$
\frac{\partial}{\partial \theta[s,a]} \log \pi(a'|s; \theta) = 
\begin{cases}
1 - \pi(a|s; \theta) & \text{if } a' = a \\
-\pi(a|s; \theta) & \text{if } a' \neq a
\end{cases}
$

##### Step 9: Vector Form

In vector form for all actions:

$
\nabla_\theta \log \pi(a|s; \theta) = \mathbf{1}_a - \pi(s; \theta)
$

Where $\mathbf{1}_a$ is a one-hot vector with 1 at position $a$.

##### Step 10: Final Update Rule

Combining everything, we get the REINFORCE update:

$
\theta \leftarrow \theta + \alpha \cdot \gamma^t \cdot G_t \cdot \left[ \mathbf{1}_{a_t} - \pi(s_t; \theta) \right]
$

##### Step 11: Monte Carlo Approximation

In practice, we approximate the expectation by sampling:

$
\theta \leftarrow \theta + \alpha \cdot \frac{1}{N} \sum_{i=1}^N \sum_{t=0}^T \gamma^t G_t^i \cdot \left[ \mathbf{1}_{a_t^i} - \pi(s_t^i; \theta) \right]
$

##### Key Insights:

1. **The dynamics disappear**: We don't need to know $P(s_{t+1}|s_t, a_t)$
2. **Only need trajectories**: We can learn from experience alone
3. **Credit assignment**: Each action gets credit for subsequent rewards
4. **Variance reduction**: Using $G_t$ instead of total return reduces variance